In [1]:
import os 

In [2]:

!pip install -qq transformers
!pip install -qq optuna
!pip install -qq datasets

In [3]:
import transformers
import datasets
from transformers import AutoTokenizer,AutoModelForQuestionAnswering, AutoModelForSequenceClassification,AdamW, get_linear_schedule_with_warmup,Trainer, TrainingArguments
from transformers import DataCollator, DataCollatorForLanguageModeling,default_data_collator
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
import random
from textwrap import wrap
from datetime import datetime
from datasets import load_from_disk
from datasets import load_dataset
from datasets import Dataset
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from torch import nn

In [4]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
MODEL_NAME = "distilbert-base-uncased"
# max sequence length for each document/sentence sample
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE= 6.58e-5
WEIGHT_DECAY = 0.289
WARMUP_STEPS = 464
RANDOM_SEED=22
LEARNING_RATE_DECAY_MULTIPLIER = 0.95
REINIT_LAYERS = 2

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [11]:

class LLRDTrainer(Trainer):

    def create_optimizer_and_scheduler(self, num_training_steps: int):
        """
        Setup the optimizer and the learning rate scheduler.
        We provide a reasonable default that works well. If you want to use something else, you can pass a tuple in the
        Trainer's init through `optimizers`, or subclass and override this method (or `create_optimizer` and/or
        `create_scheduler`) in a subclass.
        """
        self.create_optimizer()
        parameters = get_optimizer_parameters_with_llrd(self.model, LEARNING_RATE, 0.95)
        self.optimizer = AdamW(parameters, lr=LEARNING_RATE,weight_decay=WEIGHT_DECAY)
        self.create_scheduler(num_training_steps=num_training_steps, optimizer=self.optimizer)


def set_seed(seed):
    """Set all seeds to make results reproducible (deterministic mode).
       When seed is None, disables deterministic mode.
    :param seed: an integer to your choosing
    """
    if seed is not None:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        np.random.seed(seed)
        random.seed(seed)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)
  confusion_matrix = classification_report(labels, preds, digits=4,output_dict=True)
  return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'hate_f1': confusion_matrix["0"]["f1-score"],
        'hate_recall': confusion_matrix["0"]["recall"],
        'hate_precision': confusion_matrix["0"]["precision"],
        'offensive_f1': confusion_matrix["1"]["f1-score"],
        'offensive_recall': confusion_matrix["1"]["recall"],
        'offensive_precision': confusion_matrix["1"]["precision"],
        'normal_f1': confusion_matrix["2"]["f1-score"],
        'normal_recall': confusion_matrix["2"]["recall"],
        'normal_precision': confusion_matrix["2"]["precision"],    
  }

def model_init():
  return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels=3).to(device)

def timestamp():
    dateTimeObj = datetime.now()
    timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S.%f)")
    print(timestampStr)


def get_optimizer_parameters_with_llrd(model, peak_lr, multiplicative_factor):
    num_encoder_layers = len(model.distilbert.transformer.layer)
    # Task specific layer gets the peak_lr
    tsl_parameters = [
        {
            "params": [param for name, param in model.named_parameters() if 'distilbert' not in name],
            "param_names": [name for name, param in model.named_parameters() if 'distilbert' not in name],
            "lr": peak_lr,
            "name": "tsl",
        }
    ]

    # Starting from the last encoder layer each encoder layers get a lr defined by
    # current_layer_lr = prev_layer_lr * multiplicative_factor
    # the last encoder layer lr = peak_lr * multiplicative_factor
    encoder_parameters = [
        {
            "params": [param for name, param in model.named_parameters() if f"distilbert.transformer.layer.{layer_num}" in name],
            "param_names": [name for name, param in model.named_parameters() if f"distilbert.transformer.layer.{layer_num}" in name],
            "lr": peak_lr * (multiplicative_factor ** (num_encoder_layers - layer_num)),
            "name": f"layer_{layer_num}",
        }
        for layer_num, layer in enumerate(model.distilbert.transformer.layer)
    ]

    # Embedding layer gets embedding layer lr = first encoder layer lr * multiplicative_factor
    embedding_parameters = [
        {
            "params": [param for name, param in model.named_parameters() if 'embeddings' in name],
            "param_names": [name for name, param in model.named_parameters() if 'embeddings' in name],
            "lr": peak_lr * (multiplicative_factor ** (num_encoder_layers + 1)),
            "name": "embedding",
        }
    ]
    return tsl_parameters + encoder_parameters + embedding_parameters


In [6]:
set_seed(RANDOM_SEED)

In [8]:
dataset_dfs = load_from_disk('/content/drive/MyDrive/Dissertation/datasets/hatetwit_'+str(1))

In [9]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Dissertation/disbert_hate_llrd/hyper/results',          # output directory
    num_train_epochs=EPOCHS,              # total number of training epochs
    save_strategy ="epoch" ,
    per_device_train_batch_size=BATCH_SIZE,  # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,   # batch size for evaluation
    weight_decay= WEIGHT_DECAY,               # strength of weight decay
    learning_rate= LEARNING_RATE, 
    logging_dir='./disbert_hate_llrd/hyper/logs',     # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    evaluation_strategy="epoch",
    #eval_steps = 500     # evaluate each `logging_steps`
)

In [12]:
hyper_trainer = LLRDTrainer(
    model_init=model_init,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset_dfs['train'],         # training dataset
    eval_dataset=dataset_dfs['validation'],          # evaluation dataset
    compute_metrics=compute_metrics     # the callback that computes metrics of interest
)

https://huggingface.co/distilbert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpoagdoj79


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
creating metadata file for /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
creating metadata file for /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weigh

In [13]:
def hp_space_optuna(trial) :
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 3),
        "seed": trial.suggest_int("seed", 1, 40),
        "warmup_steps": trial.suggest_int("warmup_steps", 0, 500),
        "weight_decay": trial.suggest_float("weight_decay", 0, 0.3),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [ 16, 32]),
    }

In [14]:
best_trial = hyper_trainer.hyperparameter_search(n_trials=40, direction="maximize", backend="optuna", hp_space=hp_space_optuna)

[I 2022-02-13 21:32:15,668] A new study created in memory with name: no-name-e8548c78-ab27-41f9-8116-83ecf8a9f26c
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinu

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.581500,0.531052,0.776991,0.719508,0.739714,0.719972,0.716539,0.799799,0.648980,0.858341,0.871529,0.845546,0.583643,0.488589,0.724615
2,0.432900,0.493823,0.805323,0.763135,0.760917,0.767693,0.768281,0.813883,0.727518,0.874953,0.867827,0.882198,0.646170,0.621369,0.673034
3,0.291100,0.551367,0.806182,0.762200,0.762712,0.763177,0.775352,0.803823,0.748828,0.876363,0.877823,0.874908,0.634886,0.607884,0.664399


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Dissertation/disbert_hate_llrd/hyper/results/run-0/checkpoint-2330
Configuration saved in /content/drive/MyDrive/Dissertation/disbert_hate_llrd/hyper/results/run-0/checkpoint-2330/config.json
Model weights saved in /content/drive/MyDrive/Dissertation/disbert_hate_llrd/hyper/results/run-0/checkpoint-2330/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Savin

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.564800,0.524745,0.790298,0.741399,0.752521,0.732149,0.717976,0.685111,0.754153,0.865537,0.888930,0.843344,0.640683,0.622407,0.660066
2,0.425200,0.490465,0.801674,0.767268,0.758053,0.779612,0.771947,0.791751,0.753110,0.869315,0.838578,0.902390,0.660542,0.708506,0.618659
3,0.279500,0.559296,0.808328,0.768571,0.766243,0.771155,0.780296,0.796781,0.764479,0.875442,0.870418,0.880524,0.649974,0.646266,0.653725


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Dissertation/disbert_hate_llrd/hyper/results/run-1/checkpoint-2330
Configuration saved in /content/drive/MyDrive/Dissertation/disbert_hate_llrd/hyper/results/run-1/checkpoint-2330/config.json
Model weights saved in /content/drive/MyDrive/Dissertation/disbert_hate_llrd/hyper/results/run-1/checkpoint-2330/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Savin

...

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
[I 2022-02-13 23:25:29,631] Trial 37 pruned. 
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.576200,0.526576,0.790513,0.738426,0.754522,0.725963,0.725263,0.693159,0.760486,0.866465,0.899667,0.835626,0.623549,0.585062,0.667456
2,0.417300,0.499708,0.800816,0.759643,0.753968,0.766773,0.762906,0.802817,0.726776,0.872645,0.857460,0.888377,0.643379,0.640041,0.646751


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Dissertation/disbert_hate_llrd/hyper/results/run-38/checkpoint-2330
Configuration saved in /content/drive/MyDrive/Dissertation/disbert_hate_llrd/hyper/results/run-38/checkpoint-2330/config.json
Model weights saved in /content/drive/MyDrive/Dissertation/disbert_hate_llrd/hyper/results/run-38/checkpoint-2330/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
[I

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.568700,0.527016,0.786649,0.728569,0.760163,0.711768,0.722534,0.711268,0.734164,0.861587,0.912625,0.815955,0.601586,0.511411,0.730370


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
[I 2022-02-13 23:30:23,538] Trial 39 pruned. 


In [15]:
best_trial

BestRun(run_id='1', objective=10.032200592981592, hyperparameters={'learning_rate': 2.1221137870433124e-05, 'num_train_epochs': 3, 'seed': 7, 'warmup_steps': 164, 'weight_decay': 0.1290288922573804, 'per_device_train_batch_size': 16})